# Exploring data

In [1]:
import pandas as pd
import numpy as np 
import warnings; warnings.simplefilter('ignore')
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from ast import literal_eval
from matplotlib import pyplot as plt
from nltk.stem.snowball import SnowballStemmer                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                

## Simple Recommender
### Sort movie by its popularity
How to calculate movie popularity? 

According to imdb, movie rating is calculated as follow:

Weight Rated = (v/(m+v)*R) + (m/(v+m)*C)

For v = numbers of votes for the movie

For m = The minimum votes required to be on the chart

For R = The average rating of the movie 

For C = The mean vote across the whole record

In [2]:
#or use the movies_data.csv created by the DataProcessing file
movie_data = pd.read_csv("data/md.csv")
link_id = pd.read_csv("data/links_small.csv")
link_id = link_id[['movieId','imdbId', 'tmdbId']]

In [3]:
#m : minimum vote to be in the list 
m = movie_data['vote_count'].quantile(0.95)

#C: mean vote accross every record
C = movie_data['vote_average'].mean()

def weightRate(movie):
    #v : number of vote 
    v = movie['vote_count']
    #R : average rating of a movie 
    R = movie['vote_average']
    return R*v/(v+m) + C*m/(v+m)
    
movie_data['weight_rate'] = movie_data.apply(weightRate, axis = 1)
mdRanked = movie_data.sort_values(by= ['weight_rate'], ascending = False)

## Content based recommender
### What's the different 
This recommender system based on the used preference, more details of its, look at the google docs sheet 


In [4]:
tmdbId = link_id[link_id['tmdbId'].notnull()]['tmdbId']
movie_data = movie_data[movie_data['id'].notnull()]

In [5]:
tmdb_md = movie_data[movie_data['id'].isin(tmdbId)]
tmdb_md['tagline'] = tmdb_md['tagline'].fillna('')
tmdb_md['overview'] = tmdb_md['overview'].fillna('')
tmdb_md['description'] = tmdb_md['tagline'] + tmdb_md['overview']

tmdb_md.shape

(7252, 12)

In [6]:
small_description = tmdb_md['description']
    
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(small_description)
tfidf_matrix.shape

(7252, 219051)

In [7]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [8]:
tmdb_md = tmdb_md.reset_index()

In [9]:
titles = tmdb_md['title']
    
indices = pd.Series(data = tmdb_md.index, index =titles )

def getRecommendation(title):
    title_index = indices[title]
    sim_score = list(enumerate(cosine_sim[title_index]))
    sim_score = sorted(sim_score, key = lambda x: x[1], reverse = True)
    sim_score = sim_score[1:31]
    movie_indice = [i[0] for i in sim_score]
    
    return indices.iloc[movie_indice] 

In [10]:
getRecommendation('The Dark Knight')    

title
The Dark Knight Rises                        91
Batman Forever                             7145
Batman Returns                              994
Batman: Under the Red Hood                  615
Batman: The Dark Knight Returns, Part 2     513
Batman                                      492
Batman: Year One                           1506
Batman: Mask of the Phantasm               1400
JFK                                         613
Batman: The Dark Knight Returns, Part 1     624
Sherlock Holmes: A Game of Shadows          407
Batman Begins                               128
Q & A                                      4710
Law Abiding Citizen                         416
The File on Thelma Jordon                  5452
Flying Tigers                              5567
Night Falls on Manhattan                   4653
The Young Savages                          5842
Batman v Superman: Dawn of Justice         4233
Batman & Robin                             7250
The Rookie                        

In [11]:
credits = pd.read_csv("data/credits.csv")
keywords = pd.read_csv("data/keywords.csv")

# credits['id'] = credits['id'].astype(int)
# keywords['id'] = keywords['id'].astype(int)
# movie_data['id'] = movie_data['id'].astype(int)

In [12]:
movie_data =pd.merge(movie_data, credits, how = "inner", on= 'id')
movie_data = pd.merge(movie_data, keywords,how = "inner", on = 'id')

In [13]:
movie_data['cast'] = movie_data['cast'].apply(literal_eval)
movie_data['crew'] = movie_data['crew'].apply(literal_eval)
movie_data['keywords'] = movie_data['keywords'].apply(literal_eval)

In [14]:
def getActor(x):
    if (len(x) >5):
        x = x[:5]
    else:
        x = x
    return [actor['name'].replace(" ","") for actor in x ]

def getDirector(x):
    return [crew['name'].replace(" ", "") for crew in x if crew['job'] == 'Director']

def getKeywords(x):
    return [key['name'] for key in x]


tmdb_movie = movie_data[movie_data['id'].isin(tmdbId)]
tmdb_movie['actors'] =tmdb_movie['cast'].apply(getActor)
tmdb_movie['director'] = tmdb_movie['crew'].apply(getDirector)
tmdb_movie['keywords'] = tmdb_movie['keywords'].apply(getKeywords)

In [15]:
stemmer = SnowballStemmer('english')
stemmer.stem('dogs')
tmdb_movie['keywords'] = tmdb_movie['keywords'].apply(lambda x: [stemmer.stem(i) for i in x])

In [16]:
tmdb_movie['details'] = tmdb_movie['keywords'] + tmdb_movie['director'] + tmdb_movie['actors']
tmdb_movie['details'] = tmdb_movie['details'].apply (lambda x: ' '.join(x))
# tmdb_movie.to_csv('tmdb_movie.csv')

In [17]:
tf_details = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tf_matrix = tf.fit_transform(tmdb_movie['details'])
cosine_sim = linear_kernel(tf_matrix, tf_matrix)
cosine_sim.shape

(7880, 7880)

In [18]:
tmdb_movie = tmdb_movie.reset_index()

In [19]:
titles = tmdb_movie['title']    
indices = pd.Series(data = tmdb_movie.index, index =titles )

In [20]:
getRecommendation('The Dark Knight')

title
Batman Begins                               132
The Dark Knight Rises                        95
Batman: Under the Red Hood                  623
Batman Returns                             1010
Batman                                      500
Batman & Robin                             7878
Superman                                    785
Batman v Superman: Dawn of Justice         4549
Superman III                               7609
Justice League: Crisis on Two Earths       2300
Man of Steel                                928
Wonder Woman                               2358
Batman: Mask of the Phantasm               1512
Batman: Year One                           1618
Batman Forever                             7773
Kick-Ass 2                                 1561
Batman: The Dark Knight Returns, Part 2     521
All Star Superman                          2887
Superman II                                1692
Batman: The Dark Knight Returns, Part 1     632
Justice League: The Flashpoint Par